## Analyses scripts for the manuscript "Cultural Individualism/Collectivism Moderates the Effects of Climate Action Interventions"

### Install Libraries

In [270]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import anova
import bff
import matplotlib.pyplot as plt
plt.rcdefaults()
import seaborn as sns 
sns.set(style="ticks", color_codes=True)
sns.set_style("white")
sns.set_style({'xtick.bottom': True, 'ytick.left': True})
colorref = ["gray", "royalblue", "crimson", "goldenrod", "mediumorchid", "seagreen"]

# iPython magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2
%autosave 30

SMALL_SIZE = 12
MEDIUM_SIZE = 12
BIG_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIG_SIZE)  # fontsize of the figure title
cust_palette = sns.color_palette("Paired")[6:10]
cust_palette = [cust_palette[i] for i in [1,0,3,2]]

def median_split(S):
    return S > S.median()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 30 seconds


### Import data and run data preprocessing

In [271]:
df = pd.read_excel("data63.xlsx")
df

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,Edu,Income,Indirect_SES,MacArthur_SES,PerceivedSciConsensu_1,Intro_Timer,condition_time_total,WEPTcc,BELIEFcc,POLICYcc
0,R_1r7Yr7R1xMimIvj,usa,5,CollectAction,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,98.0,77.0,86.0,93.0,71.0,...,3.0,5.0,"2,4,5",7.0,80.0,6.056,95.501,1,88.50,89.000000
1,R_2ATgV5VxewrTRiI,usa,1,Control,PolicySupport|BeliefinAnthrCC|Socialmediashari...,49.0,50.0,51.0,51.0,1.0,...,3.0,4.0,"1,2,3,4,5,6,7",5.0,51.0,16.284,200.742,0,50.25,66.222222
2,R_3ENlePBlLyYjExr,usa,9,LetterFutureGen,SocialmediasharingPipedText|PolicySupport|Beli...,87.0,82.0,84.0,83.0,84.0,...,3.0,5.0,"1,2,3,6,7",8.0,73.0,5.701,218.442,1,84.00,84.222222
3,R_2c0HQgvYzas6mus,usa,4,SciConsens,PolicySupport|SocialmediasharingPipedText|Beli...,1.0,1.0,0.0,0.0,1.0,...,3.0,5.0,"1,2,3,4,5,6,7",4.0,1.0,13.282,12.702,0,0.50,1.222222
4,R_00siryCwgdMus6J,usa,8,PluralIgnorance,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,56.0,59.0,61.0,54.0,79.0,...,3.0,5.0,"1,3,5,6,7",7.0,65.0,8.474,19.315,0,57.50,80.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,3.0,2.0,"1,2,3,4,5,6,7",7.0,60.0,27.732,873.538,0,80.75,82.500000
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,3.0,2.0,"1,2,4,6,7",3.0,74.0,49.502,0.000,0,96.50,86.444444
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,3.0,1.0,"2,4,5,6,7",6.0,100.0,7.920,89.072,7,100.00,83.333333
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,3.0,NaN,"1,2,3,4,5,6,7",7.0,100.0,40.222,398.193,8,64.75,66.666667


In [272]:
# Global median split
# df["Ideology"] = (df["Politics2_1"] > df["Politics2_1"].median()).astype(int)

df['ide'] = (df['Politics2_1'] + df['Politics2_9'])/2

# Per-country median split
new = []
for _, sub in df.groupby("Country"):
    sub.loc[:, "Ideology"] = (sub.loc[:, "ide"] > sub.loc[:, "ide"].median()).astype(float)
    # Ensure NaN ideology when Politics2_1 is NaN
    sub.loc[sub.Politics2_1.isna(), "Ideology"] = np.NaN
    new.append(sub)
df = pd.concat(new)
# Replace 1 with "Conservative" and 0 with "Liberal"
df['Ideology'] = df['Ideology'].replace(1, 'Conservative').replace(0, 'Liberal')

In [273]:
#load predictor dataframe
predictordata = pd.read_excel("PredictorData.xlsx")
predictordata

,Country,GDP,CRI,EPI,PostmaterialistIndex,PowerDistanceHI,IndividualismHI,MasculinityHI,UncertaintyAvoidanceHI,LongTermOrientationHI,IndulgenceHI,Notes
0,algeria,163.138,105.17,29.6,2.00,80.0,35,35.0,70.0,26.0,32.0,NaN
1,armenia,13.861,98.17,48.3,1.19,85.0,22,50.0,88.0,61.0,25.0,NaN
2,australia,1701.893,28.00,60.1,1.95,38.0,90,61.0,51.0,21.0,71.0,NaN
3,austria,471.685,56.83,66.5,NaN,11.0,55,79.0,70.0,60.0,63.0,NaN
4,azerbaijan,54.622,118.00,38.6,NaN,85.0,22,50.0,88.0,61.0,22.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
59,unitedarabemirates,415.022,118.00,52.4,NaN,74.0,36,52.0,66.0,22.0,22.0,NaN
60,uruguay,59.321,79.67,37.4,2.45,61.0,36,38.0,98.0,26.0,53.0,NaN
61,usa,25464.475,NaN,51.1,1.94,40.0,91,62.0,46.0,26.0,68.0,NaN
62,venezuela,102.020,118.00,46.4,NaN,81.0,12,73.0,76.0,16.0,100.0,NaN


In [274]:
#merge main dataframe with predictor data & wvs survey data
df = df.merge(predictordata, on='Country', how='left')
df 

,ResponseId,Country,cond,condName,Intervention_order,Belief1,Belief2,Belief3,Belief4,Policy1,...,CRI,EPI,PostmaterialistIndex,PowerDistanceHI,IndividualismHI,MasculinityHI,UncertaintyAvoidanceHI,LongTermOrientationHI,IndulgenceHI,Notes
0,R_1DNJsF7DGNuIb9e,algeria,4,SciConsens,PolicySupport|BeliefinAnthrCC|Socialmediashari...,50.0,50.0,53.0,53.0,66.0,...,105.17,29.6,2.0,80.0,35.0,35.0,70.0,26.0,32.0,NaN
1,R_3KOO2oUWQNtKzrD,algeria,8,PluralIgnorance,PolicySupport|SocialmediasharingPipedText|Beli...,100.0,100.0,100.0,100.0,1.0,...,105.17,29.6,2.0,80.0,35.0,35.0,70.0,26.0,32.0,NaN
2,R_3OdqhjO2quQKaIx,algeria,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,93.0,67.0,100.0,33.0,15.0,...,105.17,29.6,2.0,80.0,35.0,35.0,70.0,26.0,32.0,NaN
3,R_2QuK4DRmnWL9FXS,algeria,10,DynamicNorm,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,NaN,3.0,NaN,NaN,0.0,...,105.17,29.6,2.0,80.0,35.0,35.0,70.0,26.0,32.0,NaN
4,R_1DYkviJgtp5u6JV,algeria,7,PsychDistance,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,68.0,100.0,100.0,95.0,70.0,...,105.17,29.6,2.0,80.0,35.0,35.0,70.0,26.0,32.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59435,R_2TuXye2HP6EO5KZ,vietnam,11,FutureSelfCont,PolicySupport|SocialmediasharingPipedText|Beli...,49.0,100.0,100.0,74.0,70.0,...,50.17,20.1,NaN,70.0,20.0,40.0,30.0,57.0,35.0,NaN
59436,R_3R3p8C0gzLKTcCu,vietnam,12,BindingMoral,BeliefinAnthrCC|SocialmediasharingPipedText|Po...,94.0,98.0,98.0,96.0,92.0,...,50.17,20.1,NaN,70.0,20.0,40.0,30.0,57.0,35.0,NaN
59437,R_OeT1JV3yC1aJ4EF,vietnam,8,PluralIgnorance,SocialmediasharingPipedText|PolicySupport|Beli...,100.0,100.0,100.0,100.0,80.0,...,50.17,20.1,NaN,70.0,20.0,40.0,30.0,57.0,35.0,NaN
59438,R_2azVf0KXD3rtly8,vietnam,3,NegativeEmotions,SocialmediasharingPipedText|BeliefinAnthrCC|Po...,79.0,51.0,78.0,51.0,0.0,...,50.17,20.1,NaN,70.0,20.0,40.0,30.0,57.0,35.0,NaN


In [275]:
df = df.rename(columns={"IndividualismHI": "Ind_score"})
#create variable split around 50
df['Ind_ms'] = df['Ind_score'].apply(lambda x: 1 if x < 50 else 2)

#global median split
df_col = df.loc[df['Ind_ms']==1] #dataframe for collectivist countries
df_ind = df.loc[df['Ind_ms']==2] #dataframe for individualist countries
df = df.dropna(subset=['Ind_ms'])
#59440 participants

# Demographic descriptives

In [276]:
# Calculate percentage of men and women
#1 is male, 2 is female, 4 is other
gender_counts = df['Gender'].value_counts(normalize=True) * 100
percentage_men = gender_counts.get(1, 0)  
percentage_women = gender_counts.get(2, 0) 
percentage_other = gender_counts.get(4, 0)  # Get percentage of other

# Calculate mean and standard deviation for Age
mean_age = df['Age'].mean()
sd_age = df['Age'].std()

# Printing the results
print(f"Percentage of Men: {percentage_men:.2f}%")
print(f"Percentage of Women: {percentage_women:.2f}%")
print(f"Percentage of Other: {percentage_other:.2f}%")
print(f"Mean Age: {mean_age:.2f}")
print(f"Age Standard Deviation: {sd_age:.2f}")

Percentage of Men: 47.74%
Percentage of Women: 51.56%
Percentage of Other: 0.70%
Mean Age: 39.13
Age Standard Deviation: 15.76


In [277]:
unique_countries_count = df['Country'].nunique()

print("Number of unique countries:", unique_countries_count)

Number of unique countries: 63


In [278]:
#Calculate cronbachs alphas
# List of columns that are part of the BELIEFcc scale
belief_items = ['Belief1', 'Belief2', 'Belief3', 'Belief4'] 

# Function to calculate Cronbach's Alpha
def cronbach_alpha(df, items):
    df_items = df[items]
    item_variances = df_items.var(axis=0, ddof=1)
    total_var = df_items.sum(axis=1).var(ddof=1)
    n_items = len(items)
    return (n_items / (n_items - 1)) * (1 - (item_variances.sum() / total_var))

# Calculate Cronbach's Alpha for BELIEFcc
alpha = cronbach_alpha(df, belief_items)
print(f"Cronbach's Alpha for BELIEFcc: {alpha}")


Cronbach's Alpha for BELIEFcc: 0.9463829584688861


In [279]:
#Calculate cronbachs alphas
# List of columns that are part of the POLICYcc scale
policy_items = ['Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9'] 

# Function to calculate Cronbach's Alpha
def cronbach_alpha(df, items):
    df_items = df[items]
    item_variances = df_items.var(axis=0, ddof=1)
    total_var = df_items.sum(axis=1).var(ddof=1)
    n_items = len(items)
    return (n_items / (n_items - 1)) * (1 - (item_variances.sum() / total_var))

# Calculate Cronbach's Alpha for POLICYcc
alpha = cronbach_alpha(df, policy_items)
print(f"Cronbach's Alpha for POLICYcc: {alpha}")

Cronbach's Alpha for POLICYcc: 0.9098078362900452


## Main manuscript analyses

In [280]:
import warnings
warnings.filterwarnings('ignore')

# Load R in Python
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [281]:
%R library(lme4)
%R library(lmerTest)
%R library(ordinal)
%R library(performance) #for calculating R2

array(['openxlsx', 'performance', 'MuMIn', 'effectsize', 'ordinal',
       'lmerTest', 'lme4', 'Matrix', 'tools', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'], dtype='<U11')

# 1 Do climate beliefs and behaviors differ across the individualism/collectivism spectrum?

In [282]:
#create control condition dataframe
dfc = df[df['condName'] == "Control"]
dfc['condName'] = dfc['condName'].replace(['Control'], 'aControl')
#5085 rows

# 1.1 Climate Belief

In [283]:
dfc_belief = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', 'Ind_ms', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 'Ideology', 'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', 'Ind_ms', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 'Ideology'],
    var_name="Item",
    value_name="Belief"
)
#4769 participants

In [284]:
#deal with missing data for key outcome variable
dfc_belief = dfc_belief.dropna(subset=["Belief"])

%Rpush dfc_belief

In [285]:
%%R

M1 <- lmer(Belief ~ Ind_score + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ind_score + (1 | Country) + (1 | ResponseId) + (1 |  
    Item)
   Data: dfc_belief

REML criterion at convergence: 171620.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.0436 -0.2857  0.0931  0.3180  5.6654 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 517.3986 22.7464 
 Country    (Intercept)  26.6870  5.1659 
 Item       (Intercept)   0.2871  0.5358 
 Residual               159.5155 12.6299 
Number of obs: 20015, groups:  ResponseId, 5011; Country, 62; Item, 4

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)    
(Intercept) 86.57404    1.71580 67.47562  50.457  < 2e-16 ***
Ind_score   -0.14498    0.03262 56.48924  -4.444 4.17e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
          (Intr)
Ind_score -0.880
# R2 for Mixed Models

  Conditional R

In [286]:
%%R

#standardize variables
dfc_belief$Income <- scale(dfc_belief$Income)
dfc_belief$Edu <- scale(dfc_belief$Edu)
dfc_belief$Age <- scale(dfc_belief$Age)
dfc_belief$ide <- scale(dfc_belief$ide)
dfc_belief$GDP <- scale(dfc_belief$GDP)

M1 <- lmer(Belief ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_belief)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_belief

REML criterion at convergence: 127217.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2866 -0.2872  0.0890  0.3210  5.9492 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 500.0621 22.3621 
 Country    (Intercept)  19.1997  4.3817 
 Item       (Intercept)   0.3104  0.5572 
 Residual               147.5534 12.1472 
Number of obs: 14958, groups:  ResponseId, 3742; Country, 59; Item, 4

Fixed effects:
                       Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)            82.75428    1.90524   69.95633  43.435  < 2e-16 ***
Ind_score              -0.11805    0.03198   58.08111  -3.692 0.000493 ***
Income                  1.00840    0.44411 2422.16587   2.271 0.023257 *  
Edu

# 1.2 Climate Policy Support

In [287]:
dfc_policy = pd.melt(
    dfc.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', 'Ind_ms', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 'Ideology', 'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', 'Ind_ms', "Income", "Edu", "Age", "Gender", "GDP", 'ide', 'Ideology'],
    var_name="Item",
    value_name="Policy"
)

#deal with missing data
dfc_policy = dfc_policy.dropna(subset=["Policy"])

%Rpush dfc_policy

In [288]:
%%R

M1 <- lmer(Policy ~ Ind_score + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ind_score + (1 | Country) + (1 | ResponseId) + (1 |  
    Item)
   Data: dfc_policy

REML criterion at convergence: 396770.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5698 -0.5259  0.0684  0.6332  4.0948 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 309.73   17.599  
 Country    (Intercept)  14.99    3.872  
 Item       (Intercept) 150.29   12.259  
 Residual               434.17   20.837  
Number of obs: 43407, groups:  ResponseId, 4980; Country, 62; Item, 9

Fixed effects:
            Estimate Std. Error      df t value Pr(>|t|)    
(Intercept)  74.8101     4.2879  9.6758  17.447 1.24e-08 ***
Ind_score    -0.1024     0.0249 60.0474  -4.112 0.000121 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
          (Intr)
Ind_score -0.270
# R2 for Mixed Models

  Conditional R2: 

In [289]:
%%R
#standardize variables
dfc_policy$Income <- scale(dfc_policy$Income)
dfc_policy$Edu <- scale(dfc_policy$Edu)
dfc_policy$Age <- scale(dfc_policy$Age)
dfc_policy$ide <- scale(dfc_policy$ide)
dfc_policy$GDP <- scale(dfc_policy$GDP)

M1 <- lmer(Policy ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = dfc_policy)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: dfc_policy

REML criterion at convergence: 298417

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6011 -0.5260  0.0624  0.6276  4.2189 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 314.39   17.731  
 Country    (Intercept)  13.46    3.669  
 Item       (Intercept) 146.43   12.101  
 Residual               406.88   20.171  
Number of obs: 32856, groups:  ResponseId, 3735; Country, 59; Item, 9

Fixed effects:
                      Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)            73.6862     4.3273   10.5519  17.028 5.16e-09 ***
Ind_score              -0.0963     0.0266   58.5553  -3.620 0.000615 ***
Income                  0.4472     0.3647 2454.5471   1.226 0.220245    
Edu          

# 1.3 Sharing

In [290]:
#deal with missing data
dfc_share = dfc.dropna(subset=["SHAREcc"])

%Rpush dfc_share

In [291]:
%%R

M1 <- glmer(SHAREcc ~ Ind_score + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ Ind_score + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  4776.0   4794.8  -2385.0   4770.0     3884 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5748 -0.7507 -0.3678  0.8169  3.1562 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 1.098    1.048   
Number of obs: 3887, groups:  Country, 62

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.949605   0.306569   3.098 0.001951 ** 
Ind_score   -0.020534   0.006038  -3.401 0.000672 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
          (Intr)
Ind_score -0.888
# R2 for Mixed Models

  Conditional R2: 0.294
     Marginal R2: 0.059


In [292]:
%%R

#standardize variables
dfc_share$Income <- scale(dfc_share$Income)
dfc_share$Edu <- scale(dfc_share$Edu)
dfc_share$Age <- scale(dfc_share$Age)
dfc_share$ide <- scale(dfc_share$ide)
dfc_share$GDP <- scale(dfc_share$GDP)

M1 <- glmer(SHAREcc ~ Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country), family = binomial, data = dfc_share)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country)
   Data: dfc_share

     AIC      BIC   logLik deviance df.resid 
  3643.2   3709.1  -1810.6   3621.2     2938 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1861 -0.8032  0.3732  0.7730  3.9535 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.8491   0.9215  
Number of obs: 2949, groups:  Country, 59

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)           1.095059   0.330398   3.314 0.000919 ***
Ind_score            -0.016430   0.005721  -2.872 0.004083 ** 
Income               -0.024296   0.050393  -0.482 0.629720    
Edu                   0.128278   0.044511   2.882 0.003952 ** 
Age                  -0.254928   0.045049  -5.659 1.52e-08 ***
as.factor(Gender)2   -0.2875

# 1.4 Behavior

In [293]:
# deal with missing data
dfc_WEPT = dfc.dropna(subset=["WEPTcc"])

%Rpush dfc_WEPT

In [294]:
%%R

M1 <- clmm(as.factor(WEPTcc) ~ Ind_score + (1 | Country), data = dfc_WEPT, threshold = "equidistant",  doFit = TRUE)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Cumulative Link Mixed Model fitted with the Laplace approximation

formula: as.factor(WEPTcc) ~ Ind_score + (1 | Country)
data:    dfc_WEPT

 link  threshold   nobs logLik   AIC      niter    max.grad cond.H 
 logit equidistant 5062 -8095.22 16198.44 168(513) 1.40e-02 1.3e+04

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.1882   0.4338  
Number of groups:  Country 62 

Coefficients:
            Estimate Std. Error z value Pr(>|z|)
Ind_score -0.0009241  0.0026980  -0.343    0.732

Threshold coefficients:
             Estimate Std. Error z value
threshold.1 -1.377488   0.141173  -9.757
spacing      0.187772   0.004336  43.307
(23 observations deleted due to missingness)
# R2 for Mixed Models

  Conditional R2: 0.054
     Marginal R2: 0.000


# 2. How do climate interventions impact the conceptual and behavioral outcomes of people in countries along the individualism spectrum?

## 2.1 Interventions’ Impact on Beliefs

In [295]:
df_belief = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "ide", 'Belief1', 'Belief2', 'Belief3', 'Belief4']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", "Income", "Edu", "Age", "Gender", "GDP", "ide"],
    var_name="Item",
    value_name="Belief"
)
df_belief['condName'] = df_belief['condName'].replace(['Control'], 'aControl')

In [296]:
#deal with missing data
df_belief = df_belief.dropna(subset=["Belief"])
%Rpush df_belief

In [297]:
%%R
M1 <- lmer(Belief ~ condName*Ind_score + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_belief)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Belief ~ condName * Ind_score + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: df_belief

REML criterion at convergence: 1982366

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2906 -0.2693  0.0885  0.2938  6.0000 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 502.0822 22.4072 
 Country    (Intercept)  26.6574  5.1631 
 Item       (Intercept)   0.2813  0.5303 
 Residual               147.4639 12.1435 
Number of obs: 233116, groups:  ResponseId, 58370; Country, 62; Item, 4

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         8.725e+01  1.643e+00  1.022e+02  53.106
condNameBindingMoral               -5.184e-01  1.122e+00  5.825e+04  -0.462
condNameCollectAction               6.224e-01  1.118e+00  5.824e+04   0.557
condNameDynamicNorm                -4.526


Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it



In [298]:
%%R

#standardize variables
df_belief$Income <- scale(df_belief$Income)
df_belief$Edu <- scale(df_belief$Edu)
df_belief$Age <- scale(df_belief$Age)
df_belief$ide <- scale(df_belief$ide)
df_belief$GDP <- scale(df_belief$GDP)

M1 <- lmer(Belief ~ condName*Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_belief)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
Belief ~ condName * Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_belief

REML criterion at convergence: 1509872

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.5396 -0.2774  0.0835  0.3020  6.2440 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 478.7673 21.8808 
 Country    (Intercept)  20.2067  4.4952 
 Item       (Intercept)   0.3876  0.6226 
 Residual               137.8344 11.7403 
Number of obs: 178878, groups:  ResponseId, 44755; Country, 59; Item, 4

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         8.471e+01  1.779e+00  1.052e+02  47.615
condNameBindingMoral                3.700e-01  1.303e+00  4.466e+04   0.284
condNameCollectAction               6.234e-01  1.305e+00  4.467


Correlation matrix not shown by default, as p = 32 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

In addition: Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.361002 (tol = 0.002, component 1)


## 2.2 Interventions’ Impact on Policy Support.

In [299]:
df_policy = pd.melt(
    df.loc[:, ['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "ide",'Policy1', 'Policy2', 'Policy3', 'Policy4','Policy5', 'Policy6', 'Policy7', 'Policy8', 'Policy9']],
    id_vars=['ResponseId', 'Country', 'condName', 'Ind_score', "Ind_ms", 'Income', 'Edu', 'Age', 'Gender', "GDP", "ide"],
    var_name="Item",
    value_name="Policy"
)
df_policy['condName'] = df_policy['condName'].replace(['Control'], 'aControl')

#deal with missing data
df_policy = df_policy.dropna(subset=["Policy"])

%Rpush df_policy

In [300]:
%%R

M1 <- lmer(Policy ~ condName*Ind_score + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_policy)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: Policy ~ condName * Ind_score + (1 | Country) + (1 | ResponseId) +  
    (1 | Item)
   Data: df_policy

REML criterion at convergence: 4615950

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.7684 -0.5203  0.0709  0.6338  4.6549 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 307.22   17.528  
 Country    (Intercept)  17.79    4.218  
 Item       (Intercept) 140.49   11.853  
 Residual               429.63   20.728  
Number of obs: 505706, groups:  ResponseId, 57970; Country, 62; Item, 9

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         7.569e+01  4.167e+00  9.854e+00  18.164
condNameBindingMoral                5.684e-01  9.172e-01  5.787e+04   0.620
condNameCollectAction               2.196e+00  9.131e-01  5.783e+04   2.405
condNameDynamicNorm                 1.191


Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it



In [301]:
%%R

#standardize variables
df_policy$Income <- scale(df_policy$Income)
df_policy$Edu <- scale(df_policy$Edu)
df_policy$Age <- scale(df_policy$Age)
df_policy$ide <- scale(df_policy$ide)
df_policy$GDP <- scale(df_policy$GDP)

M1 <- lmer(Policy ~ condName*Ind_score + Income + Edu + Age + as.factor(Gender) + ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item), data = df_policy)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: 
Policy ~ condName * Ind_score + Income + Edu + Age + as.factor(Gender) +  
    ide + GDP + (1 | Country) + (1 | ResponseId) + (1 | Item)
   Data: df_policy

REML criterion at convergence: 3556360

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9194 -0.5235  0.0634  0.6307  4.8265 

Random effects:
 Groups     Name        Variance Std.Dev.
 ResponseId (Intercept) 303.20   17.413  
 Country    (Intercept)  17.52    4.186  
 Item       (Intercept) 139.38   11.806  
 Residual               403.38   20.084  
Number of obs: 392147, groups:  ResponseId, 44605; Country, 59; Item, 9

Fixed effects:
                                     Estimate Std. Error         df t value
(Intercept)                         7.570e+01  4.240e+00  1.078e+01  17.853
condNameBindingMoral                6.256e-01  1.078e+00  4.452e+04   0.580
condNameCollectAction               2.706e+00  1.078e+00  4.450


Correlation matrix not shown by default, as p = 32 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it



## 2.3 Interventions’ Impact on Social Media Sharing.

In [302]:
#deal with missing data
df_share = df.dropna(subset=["SHAREcc"])

df_share['condName'] = df_share['condName'].replace(['Control'], 'aControl')

%Rpush df_share

In [236]:
%%R

M1 <- glmer(SHAREcc ~ condName*Ind_score + (1 | Country), family = binomial, data = df_share)
print(summary(M1))

rsq <- r2(M1) #marginal and conditional R² values
print(rsq)

write.csv(coef(summary(M1)), "coefficients_share.csv") #export coefficients to csv

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: SHAREcc ~ condName * Ind_score + (1 | Country)
   Data: df_share

     AIC      BIC   logLik deviance df.resid 
 51806.8  52023.9 -25878.4  51756.8    43705 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2713 -0.7429  0.3769  0.7827  2.5033 

Random effects:
 Groups  Name        Variance Std.Dev.
 Country (Intercept) 0.9792   0.9895  
Number of obs: 43730, groups:  Country, 62

Fixed effects:
                                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)                         0.970602   0.280134   3.465 0.000531 ***
condNameBindingMoral                0.331963   0.124538   2.666 0.007686 ** 
condNameCollectAction               0.709041   0.127913   5.543 2.97e-08 ***
condNameDynamicNorm                 0.563119   0.126577   4.449 8.63e-06 ***
condNameFutureSelfCont              0.457182   0.132704   3.445 0.000


Correlation matrix not shown by default, as p = 24 > 12.
Use print(summary(M1), correlation=TRUE)  or
    vcov(summary(M1))        if you need it

In addition: Warning messages:
1: In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model failed to converge with max|grad| = 0.0225676 (tol = 0.002, component 1)
2: In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  :
  Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?


## 2.4 Interventions’ Impact on Behaviors. 

In [237]:
#deal with missing data
df_WEPT = df.dropna(subset=["WEPTcc"])

df_WEPT['condName'] = df_WEPT['condName'].replace(['Control'], 'aControl')

%Rpush df_WEPT

In [238]:
%%R

M <- clmm(as.factor(WEPTcc) ~ condName*Ind_score + (1|Country), data = df_WEPT, threshold = "equidistant")
summary(M)

rsq <- r2(M) #marginal and conditional R² values
print(rsq)

# R2 for Mixed Models

  Conditional R2: 0.071
     Marginal R2: 0.004


In addition: Warning message:
(2) Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables? 
In addition: Absolute and relative convergence criteria were met 
